# Memoria del proyecto — Predicción de enfermedad cardíaca (Heart Failure Prediction)

**Autor/a:** Alex Ellakuria
**Fecha:** 2025-12-26  

Este notebook funciona como **memoria ejecutable** del proyecto: recoge el objetivo, los datos, el EDA, el preprocesado, el entrenamiento de modelos, la evaluación y las conclusiones, con un enfoque claro y defendible a nivel de máster.

---

## 1. Objetivo del proyecto

El objetivo es **predecir el riesgo de enfermedad cardíaca** (`enfermedad_cardiaca`) a partir de variables clínicas y demográficas.

- **Tipo de problema:** Clasificación binaria.
- **Variable objetivo:** `enfermedad_cardiaca` (1 = enfermedad, 0 = no enfermedad).
- **Uso esperado (impacto):** priorización de pacientes para cribado/seguimiento clínico y apoyo a decisiones médicas (no sustitución del diagnóstico).

---

## 2. Dataset

Se utiliza el dataset **Heart Failure Prediction** (Kaggle / UCI combinados), con aproximadamente **918 observaciones** tras limpieza del autor original.

### Variables (resumen)
- Numéricas: edad, presión en reposo, colesterol, frecuencia cardiaca máxima, oldpeak...
- Categóricas: sexo, tipo de dolor en el pecho, ECG en reposo, angina inducida por ejercicio, pendiente ST...
- Binaria: glucosa en ayunas > 120 mg/dl

In [ ]:
# =========================
# 0) Imports + configuración
# =========================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix,
    RocCurveDisplay, PrecisionRecallDisplay
)

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_STATE = 42
DATA_PATH = "data/heart.csv"
TARGET_RAW = "HeartDisease"

sns.set_theme(style="whitegrid")
plt.rcParams.update({
    "figure.dpi": 110,
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10
})

---

## 3. Carga, renombrado y diccionario de etiquetas

Trabajo con nombres de columnas en español para mejorar claridad y consistencia en el informe y los gráficos.
Además, uso un diccionario `LABELS` **solo para etiquetar gráficos** (sin cambiar datos).

In [ ]:
# =========================
# 1) Carga + renombrado
# =========================
df_raw = pd.read_csv(DATA_PATH)

RENAME = {
    "Age": "edad",
    "Sex": "sexo",
    "ChestPainType": "tipo_dolor_pecho",
    "RestingBP": "presion_reposo",
    "Cholesterol": "colesterol",
    "FastingBS": "glucosa_ayunas",
    "RestingECG": "ecg_reposo",
    "MaxHR": "fc_max",
    "ExerciseAngina": "angina_ejercicio",
    "Oldpeak": "oldpeak",
    "ST_Slope": "pendiente_st",
    "HeartDisease": "enfermedad_cardiaca"
}
df = df_raw.rename(columns=RENAME).copy()
TARGET = RENAME.get(TARGET_RAW, TARGET_RAW)

LABELS = {
    "edad": "Edad del paciente (años)",
    "sexo": "Sexo",
    "tipo_dolor_pecho": "Tipo de dolor en el pecho",
    "presion_reposo": "Presión arterial en reposo (mm Hg)",
    "colesterol": "Colesterol sérico (mg/dl)",
    "glucosa_ayunas": "Glucosa en ayunas > 120 mg/dl",
    "ecg_reposo": "Electrocardiograma en reposo",
    "fc_max": "Frecuencia cardíaca máxima alcanzada",
    "angina_ejercicio": "Angina inducida por ejercicio",
    "oldpeak": "Depresión del segmento ST (oldpeak)",
    "pendiente_st": "Pendiente del segmento ST",
    "enfermedad_cardiaca": "Enfermedad cardíaca"
}

def label(col: str) -> str:
    return LABELS.get(col, col)

df.head()

---

## 4. Split train/test (evitar leakage)

Primero divido en entrenamiento y test **antes** de decisiones de preprocesado/modelado.  
Esto hace que las conclusiones sean defendibles y evita contaminación del test.

In [ ]:
X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=RANDOM_STATE
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()

---

## 5. EDA (resumen ejecutable)

En esta sección recojo lo esencial del EDA:
- Distribución del target
- Revisión de numéricas (distribución por clase)
- Correlación numérica (solo para exploración)

> Nota: La correlación Pearson no “mide importancia”, pero ayuda a ver relaciones lineales y redundancias.

In [ ]:
categorical_cols = ["sexo", "tipo_dolor_pecho", "ecg_reposo", "angina_ejercicio", "pendiente_st"]
binary_cols = ["glucosa_ayunas"]
numeric_cols = ["edad", "presion_reposo", "colesterol", "fc_max", "oldpeak"]

# Distribución del target
plt.figure(figsize=(5,4))
sns.countplot(x=y_train)
plt.title("Distribución del target en train")
plt.xlabel(label(TARGET))
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()

In [ ]:
# Correlación (numéricas + binaria + target) en train
num_df = pd.concat([X_train[numeric_cols + binary_cols], y_train.rename(TARGET)], axis=1).copy()

# Trato ceros sospechosos como NaN en columnas típicas (criterio clínico / plausibilidad)
for c in ["presion_reposo", "colesterol", "fc_max"]:
    if c in num_df.columns:
        num_df[c] = num_df[c].replace(0, np.nan)

corr = num_df.corr(numeric_only=True)

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt=".2f", linewidths=0.5, square=True)
plt.title("Matriz de correlación (train: numéricas + target)")
plt.tight_layout()
plt.show()

---

## 6. Preparación para Machine Learning

### 6.1. Diseño del preprocesado
- **Numéricas:** imputación simple (mediana) + escalado (útil para modelos lineales y SVM).
- **Categóricas:** One-Hot Encoding.
- **Binarias:** se tratan como numéricas (0/1).

> Mantengo todo dentro de un **Pipeline** para evitar leakage y facilitar reproducibilidad.

In [ ]:
from sklearn.impute import SimpleImputer

numeric_features = numeric_cols + binary_cols
categorical_features = categorical_cols

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), numeric_features),
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_features),
    ],
    remainder="drop"
)

---

## 7. Modelos candidatos y métricas de comparación

Comparo varios modelos con métricas adecuadas al problema:

- **ROC-AUC:** capacidad de ranking global.
- **Average Precision (PR-AUC):** útil cuando importa el positivo (riesgo).
- **F1 / Recall:** relevantes si priorizo detectar casos (sensibilidad).

> En salud suele interesar **recall alto** (reducir falsos negativos), ajustando según el contexto.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    "LogReg": LogisticRegression(max_iter=2000, random_state=RANDOM_STATE),
    "SVM_RBF": SVC(probability=True, random_state=RANDOM_STATE),
    "KNN": KNeighborsClassifier(),
    "RandomForest": RandomForestClassifier(random_state=RANDOM_STATE),
    "GradientBoosting": GradientBoostingClassifier(random_state=RANDOM_STATE),
}

def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe, "predict_proba") else None

    metrics = {
        "modelo": name,
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
    }
    if y_proba is not None:
        metrics["roc_auc"] = roc_auc_score(y_test, y_proba)
        metrics["avg_precision"] = average_precision_score(y_test, y_proba)
    else:
        metrics["roc_auc"] = np.nan
        metrics["avg_precision"] = np.nan

    return pipe, metrics

results = []
fitted = {}

for name, model in models.items():
    pipe, m = evaluate_model(name, model, X_train, y_train, X_test, y_test)
    fitted[name] = pipe
    results.append(m)

results_df = pd.DataFrame(results).sort_values(["roc_auc", "avg_precision"], ascending=False)
results_df

---

## 8. Selección del modelo final y evaluación visual

Reviso:
- ROC y PR curves
- Matriz de confusión

(Después, si el contexto lo requiere, ajusto el umbral para aumentar recall.)

In [ ]:
best_name = results_df.iloc[0]["modelo"]
best_pipe = fitted[best_name]

y_proba = best_pipe.predict_proba(X_test)[:, 1]

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title(f"ROC — {best_name}")
plt.tight_layout()
plt.show()

PrecisionRecallDisplay.from_predictions(y_test, y_proba)
plt.title(f"Precision-Recall — {best_name}")
plt.tight_layout()
plt.show()

y_pred = (y_proba >= 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Reds")
plt.title(f"Matriz de confusión (umbral 0.5) — {best_name}")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.tight_layout()
plt.show()

---

## 9. Guardado del modelo final (pickle)

Guardo el pipeline completo (preprocesado + modelo).  
Esto permite reutilizarlo directamente en producción o para inferencia.

In [ ]:
import pickle

MODEL_DIR = Path("model") / "production"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

final_model_file = MODEL_DIR / f"modelo_final_{best_name}.pkl"

with open(final_model_file, "wb") as f:
    pickle.dump(best_pipe, f)

final_model_file

---

## 10. Conclusiones

- Pipeline reproducible y sin leakage.
- Comparación de múltiples modelos con métricas adecuadas.
- Elección final basada en métricas + análisis de errores.
- Modelo exportado para reutilización.

### Próximos pasos recomendados (nivel máster)
- Validación cruzada y búsqueda de hiperparámetros (GridSearch/Randomized).
- Interpretabilidad (coeficientes / SHAP según el modelo).
- Calibración de probabilidades y ajuste de umbral según coste de errores.